In [4]:
from sklearn.linear_model import LinearRegression #线性回归
from sklearn.neighbors import KNeighborsRegressor #K近邻回归
from sklearn.tree import DecisionTreeRegressor #决策树回归
from sklearn.ensemble import RandomForestRegressor #随机森林回归
from sklearn.svm import SVR #支持向量机回归
import lightgbm as lgb  #LightGBM模型

from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error


In [ ]:
!gdown --id '17aG9Z-3r83m1_BrTuHATiHhgYvjpGmvS'
# 列出目前目錄下所有的檔案
!ls


Downloading...
From: https://drive.google.com/uc?id=17aG9Z-3r83m1_BrTuHATiHhgYvjpGmvS
To: /content/task3.zip
37.3MB [00:01, 28.3MB/s]
 data.zip	    phm_test.csv   sample_data	 task3_2.csv	   task3.zip
 description.txt    really.csv	   task3_1.csv	 task3_3.csv	   train
 final_result.csv   result.csv	  'task3(2)'	 task3_split.zip


In [ ]:
!unzip -q "task3.zip" -d "./"

replace ./task3(2)/gun0.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!gdown --id '1OlSAVpo1k7__G4ToTSVMWrnNZIIPRe1E'
# 列出目前目錄下所有的檔案
!ls
!unzip -q "data.zip" -d "./"

Downloading...
From: https://drive.google.com/uc?id=1OlSAVpo1k7__G4ToTSVMWrnNZIIPRe1E
To: /content/data.zip
239MB [00:02, 81.6MB/s]
 data.zip   sample_data  'task3(2)'   task3.zip


In [ ]:
import pandas as pd
gun0 = pd.read_csv('/content/task3(2)/gun0.csv')
gun1 = pd.read_csv('/content/task3(2)/gun1.csv')
gun2 = pd.read_csv('/content/task3(2)/gun2.csv')
gun3 = pd.read_csv('/content/task3(2)/gun3.csv')
#gun4 = pd.read_csv('/content/task3(2)/gun4.csv')
gun5 = pd.read_csv('/content/task3(2)/gun5.csv')
gun6 = pd.read_csv('/content/task3(2)/gun6.csv')
gun7 = pd.read_csv('/content/task3(2)/gun7.csv')
gun8 = pd.read_csv('/content/task3(2)/gun8.csv')
gun9 = pd.read_csv('/content/task3(2)/gun9.csv')
gun10 = pd.read_csv('/content/task3(2)/gun10.csv')
gun11 = pd.read_csv('/content/task3(2)/gun11.csv')
gun12 = pd.read_csv('/content/task3(2)/gun12.csv')


In [ ]:
frames=[gun0,gun1,gun2,gun3,gun5,gun6,gun7,gun8,gun9,gun10,gun11,gun12]
gun0 = pd.concat(frames)

In [ ]:
gun12.shape

(415725, 24)

In [ ]:
frame=['W_Error', 'area']
gun0 = gun0.drop(frame,axis=1)

In [ ]:
gun0.shape

(4512913, 22)

In [ ]:
test = pd.read_csv('/content/phm_test.csv')

In [ ]:
test.columns = ['time', 'gun_no','C_Cylinder_force','C_Differential_pressure','W_Friction','W_Maximum_aperture','W_Maximum_electrode_force',
            'W_Start_friction','W_us2','W_Welding_point_count','W_position_count','area','in_Counterbalance_pressure','in_Electrode_force','in_Electrode_position','in_Sheet_thickness',
            'in_Velocity','in_force_build_up','out_Cap_offset','out_Electrode_force','out_Electrode_position','out_force_build_up']

In [ ]:
def str2int(dt):
    errordata=dt['W_Error']
    errordata[errordata!='E016']=0
    errordata[errordata=='E016']=1
    errordata.replace(1,1,inplace=True)
    gunnum=dt['gun_no']
    gunval=pd.get_dummies(gunnum).columns
    for i in range(len(gunval)):
        gunnum.replace(gunval[i],i,inplace=True)
    return dt

In [ ]:
def ses_str2int(dt):

    gunnum=dt['gun_no']
    gunval=pd.get_dummies(gunnum).columns
    for i in range(len(gunval)):
        gunnum.replace(gunval[i],i,inplace=True)
    return dt

In [ ]:
test = test.drop('area',axis=1)
test = ses_str2int(test)

In [ ]:
gun0 = ses_str2int(gun0)

In [ ]:
test.shape

(2999999, 21)

In [ ]:
#test[test['gun_no']==0] = 432000

In [ ]:
train_target = gun0['lifetime']
train_data = gun0.drop('lifetime', axis=1)

In [ ]:
print(train_data.shape)
print(train_target.shape)

(4512913, 21)
(4512913,)


In [ ]:
train_data.columns
test.columns

Index(['time', 'gun_no', 'C_Cylinder_force', 'C_Differential_pressure',
       'W_Friction', 'W_Maximum_aperture', 'W_Maximum_electrode_force',
       'W_Start_friction', 'W_us2', 'W_Welding_point_count',
       ' W_position_count', 'in_Counterbalance_pressure', 'in_Electrode_force',
       'in_Electrode_position', 'in_Sheet_thickness', 'in_Velocity',
       'in_force_build_up', 'out_Cap_offset', 'out_Electrode_force',
       'out_Electrode_position', 'out_force_build_up'],
      dtype='object')

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999999 entries, 0 to 2999998
Data columns (total 21 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   time                        int64  
 1   gun_no                      object 
 2   C_Cylinder_force            int64  
 3   C_Differential_pressure     float64
 4   W_Friction                  int64  
 5   W_Maximum_aperture          float64
 6   W_Maximum_electrode_force   int64  
 7   W_Start_friction            int64  
 8   W_us2                       int64  
 9   W_Welding_point_count       int64  
 10  W_position_count            int64  
 11  in_Counterbalance_pressure  float64
 12  in_Electrode_force          int64  
 13  in_Electrode_position       float64
 14  in_Sheet_thickness          float64
 15  in_Velocity                 int64  
 16  in_force_build_up           int64  
 17  out_Cap_offset              float64
 18  out_Electrode_force         int64  
 19  out_Electrode_positio

In [ ]:
from sklearn import preprocessing
feature_col = [col for col in train_data.columns if col not in ['lifetime']]
print(feature_col)
#print('\n')
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler = min_max_scaler.fit(train_data[feature_col])

#按照训练集的标准对训练和测试集进行归一化
train_data_scaler = min_max_scaler.transform(train_data[feature_col])
test_data_scaler = min_max_scaler.transform(test[feature_col])
print(type(train_data_scaler))

#再将归一化的array转为dataframe格式
train_data_scaler = pd.DataFrame(train_data_scaler)
test_data_scaler = pd.DataFrame(test_data_scaler)

['time', 'gun_no', 'C_Cylinder_force', 'C_Differential_pressure', 'W_Friction', 'W_Maximum_aperture', 'W_Maximum_electrode_force', 'W_Start_friction', 'W_us2', 'W_Welding_point_count', 'W_position_count', 'in_Counterbalance_pressure', 'in_Electrode_force', 'in_Electrode_position', 'in_Sheet_thickness', 'in_Velocity', 'in_force_build_up', 'out_Cap_offset', 'out_Electrode_force', 'out_Electrode_position', 'out_force_build_up']
<class 'numpy.ndarray'>


In [ ]:
train_data, val_data, train_target, val_target = train_test_split(train_data, train_target, test_size=.2, random_state=0,shuffle=False) # test_size=.3

In [ ]:
'''
随机森林回归
'''
clf3 = RandomForestRegressor(n_estimators=500)#200课树模型
clf3.fit(train_data,train_target)
error3 = mean_squared_error(val_target,clf3.predict(val_data))
print('随机森林的误差：', error3)

In [ ]:
'''
多元线性回归
'''
clf1 = LinearRegression()
clf1.fit(train_data,train_target)
error1 = mean_squared_error(val_target,clf1.predict(val_data))
print('线性回归模型的误差：', error1)
#result1 = clf1.predict(new_test_pca)

'''
K近邻回归
'''
clf2 = KNeighborsRegressor(n_neighbors=8)# 最近的8个
clf2.fit(train_data,train_target)
error2 = mean_squared_error(val_target,clf2.predict(val_data))
print('K近邻回归模型的误差：', error2)
#result2 = clf2.predict(new_test_pca)


In [ ]:
'''
随机森林回归
'''
clf3 = RandomForestRegressor(n_estimators=200)#200课树模型
clf3.fit(train_data,train_target)
error3 = mean_squared_error(val_target,clf3.predict(val_data))
print('随机森林的误差：', error3)

In [ ]:
'''
LGB回归模型
'''
clf4 = lgb.LGBMRegressor(
    learning_rate = 0.01,
    max_depth=-1,
    n_estimators = 5000,
    boosting_type = 'gbdt',
    random_state=2019,
    objective = 'regression'
)
clf4.fit(X=train_data,y=train_target,eval_metric = 'MSE',verbose=50)
error4 = mean_squared_error(val_target,clf4.predict(val_data))
print('lightGbm的误差：', error4)
#result4 = clf4.predict(new_test_pca)
#print(result4)

lightGbm的误差： 12517041.660419773


In [ ]:
#5折交叉验证
#利用LightGBM模型对构造的新特征进行 训练和评估
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import numpy as np

folds = 5
kf = KFold(n_splits = folds, shuffle=True, random_state=2019)
mse_dict = {'train_mse':[], 'test_mse':[]}
for i, (train_index, test_index)in enumerate(kf.split(train_data)):
  #LGB 树模型
  lgb_reg = lgb.LGBMRegressor(
      learning_rate = 0.01,
      max_depth = -1,
      n_estimators = 5000,
      boosting_type = 'gbrt',
      random_state = 2019,
      objective = 'regression',

  )
  #切分训练集和测试集
  X_train_kf = train_data[train_index]
  X_test_kf = val_data[test_index]
  y_train_kf = train_target[train_index]
  y_test_kf = val_target[test_index]
  #训练模型
  lgb_reg.fit(X=X_train_kf,
              y=y_train_kf,
              eval_set=[(X_train_kf,y_train_kf),
                        (X_test_kf,y_test_kf)],
              eval_names=['train','test'],
              early_stopping_rounds=100,
              eval_metric='MSE',
              verbose=50)
  #预测
  y_train_predict = lgb_reg.predict(X_train_kf, num_iteration=lgb_reg.best_iteration_)
  y_test_predict = lgb_reg.predict(X_test_kf, num_iteration=lgb_reg.best_iteration_)
  #结果输出
  print('第{}折，训练与预测结果的MSE值'.format(i))
  train_mse = mean_squared_error(y_train_kf, y_train_predict)
  test_mse = mean_squared_error(y_test_kf, y_test_predict)
  print('train: ',train_mse)
  print('test: ',test_mse)
  mse_dict['train_mse'].append(train_mse)
  mse_dict['test_mse'].append(test_mse)

print('训练MSE\n',mse_dict['train_mse'],'\n', np.mean(mse_dict['train_mse']),'\n------')
print('测试MSE\n',mse_dict['test_mse'],'\n', np.mean(mse_dict['test_mse']), '\n-------')

KeyError: ignored

In [ ]:
'''
采用PolynomialFeatures进行特征的组合，产生新的特征变量，然后采用SGDRegressor进行拟合
'''
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
poly = PolynomialFeatures(1)
train_data_poly = poly.fit_transform(train_data)
val_data_poly = poly.transform(val_data)
new_test_poly = poly.transform(test_data_scaler)

clf5 = SGDRegressor(max_iter=1000,tol=1e-3,penalty='L2',alpha=0.0001)
clf5.fit(train_data_poly,train_target)
error5 = mean_squared_error(val_target, clf5.predict(val_data_poly))
print(error5)
#result5 = clf5.predict(new_test_pca_poly)
#在测试集上的效果很差（原因是没有删去训练集中与测试集差别很大的特征）

1.0314482100163823e+52


In [ ]:
result = clf5.predict(new_test_poly)

In [ ]:
import csv
with open('result.csv',"w")as targetFile:
  target_writer = csv.writer(targetFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  for i in range(len(result)):
    target_writer.writerow([result[i]])

In [ ]:
import numpy as np
final_result = pd.read_csv('result.csv')
final_result = final_result.astype('int64')
final_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999998 entries, 0 to 2999997
Data columns (total 1 columns):
 #   Column              Dtype
---  ------              -----
 0   432125.29897699767  int64
dtypes: int64(1)
memory usage: 22.9 MB


In [ ]:
final_result.columns = ['value']

In [ ]:
final_result[final_result['value'].values > 432000] = 432000

In [ ]:
print(final_result.head())


    value
0  428553
1  432000
2  428552
3  428552
4  432000


In [ ]:
final_result.to_csv('final_result.csv',index=None,header=None)


In [ ]:
task = pd.read_csv('2.csv')
task.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   time    1048575 non-null  int64 
 1   gun_no  1048575 non-null  object
 2   label   1048575 non-null  int64 
 3   label3  1048575 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 32.0+ MB


In [ ]:
task.columns

Index(['time', 'gun_no', 'label', 'label3'], dtype='object')

In [ ]:
new2 =task['label3']

In [ ]:
new2.shape

(1048575,)

In [ ]:
!unzip -q "task3_split.zip" -d "./"

In [ ]:
task3_1 = pd.read_csv('/content/task3_1.csv')
task3_2 = pd.read_csv('/content/task3_2.csv')
task3_3 = pd.read_csv('/content/task3_3.csv')

In [ ]:
frame=[task3_1,task3_2,task3_3]
task = pd.concat(frame)

In [ ]:
task.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   time    int64 
 1   gun_no  object
 2   label   int64 
 3   label3  int64 
dtypes: int64(3), object(1)
memory usage: 114.4+ MB


In [ ]:
really = task['label3']

In [ ]:
really.head()

0    17544
1    17543
2    17542
3    17541
4    17540
Name: label3, dtype: int64

In [ ]:
really.to_csv('really.csv',index=None,header=None)